<a href="https://colab.research.google.com/github/pietrzakkuba/computer-assisted-translation-labs/blob/main/lab_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Komputerowe wspomaganie tłumaczenia

# Zajęcia 9 - key logging

Badania nad komputerowym wspomaganiem tłumaczenia często prowadzone są przy użyciu metodologii testowania interfejsów użytkownika - UI/UX testing. Program typu CAT traktuje się wówczas jak każdy inny program komputerowy i przeprowadza testy wydajności i użyteczności.

Testy takie prowadzone są zawsze na użytkownikach końcowych, w tym przypadku - na tłumaczach. Podstawowym celem testów jest próba zaobserwowania faktycznego sposobu pracy tłumacza - które funkcje programu są przez niego wykorzystywane najczęściej, jakich innych narzędzi poza CAT-em używa on do swojej pracy, które funkcje programu działają zgodnie, a które niezgodnie z intuicją użytkownika oraz wiele innych czynników. Aby wszystkie te analizy były możliwe, konieczne jest zgromadzenie jak największej ilości danych dotyczących przebiegu testu.

Testy są przede wszystkim nagrywane. Nagrywany jest zarówno ekran komputera (screen capture), jak i sam użytkownik pracujący przy komputerze. To jednak nie wszystko - często stosuje się specjalne techniki eye-trackingu, które są w stanie określić, w który punk ekranu użytkownik aktualnie patrzy. Dane pozyskane w ten sposób używane są do analizy czasu znalezienia przez użytkownika potrzebnej mu funkcji oraz zidentyfikowania miejsc, gdzie tej funkcji poszukiwał. Można również wyznaczyć obszary ekranu, które często skupiają uwagę użytkownika. 

Dodatkowo stosuje się jeszcze jedną technikę, która jest szczególnie przydatna z punktu widzenia analizy procesu tłumaczenia. Wykonuje się pełny key logging, tj. zapisuje się każde uderzenie użytkownika w dowolny klawisz na klawiaturze wraz z precyzyjnym czasem tego uderzenia. Dane pozyskane w ten sposób pozwalają na przeprowadzenie szeregu interesujących analiz.

Zapoznajmy się najpierw z programem typu key logger:

`sudo pip3 install keyboard`

In [ ]:
import keyboard


def report_key(event):
    print(event)

keyboard.on_release(callback=report_key)
keyboard.wait('esc')
# jakiś tekst

KeyboardEvent(j up)
KeyboardEvent(a up)
KeyboardEvent(k up)
KeyboardEvent(i up)
KeyboardEvent(alt gr up)
KeyboardEvent(s up)
KeyboardEvent(space up)
KeyboardEvent(e up)
KeyboardEvent(t up)
KeyboardEvent(k up)
KeyboardEvent(s up)
KeyboardEvent(t up)
KeyboardEvent(esc up)


UWAGA! Aby uruchomić powyższy kod na Linuxie konieczne są uprawnienia administratora (pytanie poza konkursem - dlaczego?)

### Ćwiczenie 1: Wykorzystując powyższy kod napisz keylogger, który zapisuje wszystkie uderzenia w klawisze do pliku. Format pliku jest dowolny, każdy wpis musi zawierać precyzyjną godzinę uderzenia oraz uderzony klawisz. Uruchom program i przepisz paragraf dowolnie wybranego tekstu.

In [ ]:
from datetime import datetime
import keyboard

def on_key_released(event):
    file = open('plik.tsv', 'a')
    file.write(str(event.name) + '\t' + str(event.time) + '\n')
    file.close()

keyboard.on_release(callback=on_key_released)
keyboard.wait('esc') # polski samolot myślwiski konstruckji inzyniera zygmunta pulawksiego z ikresu przed ii wojnyy swiatowej produkowany w polsce w zakładach pzl w latach 1934-1936, a tkaże na licencnji w zakładach IAR w Rumunii.

Traceback (most recent call last):
  File "C:\Users\Kuba\AppData\Roaming\Python\Python39\site-packages\keyboard\_generic.py", line 22, in invoke_handlers
    if handler(event):
  File "C:\Users\Kuba\AppData\Roaming\Python\Python39\site-packages\keyboard\__init__.py", line 480, in <lambda>
    return hook(lambda e: e.event_type == KEY_DOWN or callback(e), suppress=suppress)
  File "<ipython-input-19-2a5f4a996f30>", line 6, in on_key_released
    file.write(str(event.name) + '\t' + str(event.time) + '\n')
  File "C:\Program Files\Python39\lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
UnicodeEncodeError: 'charmap' codec can't encode character '\u015b' in position 0: character maps to <undefined>


Celem powyższego ćwiczenia jest pozyskanie danych testowych. Dalsze analizy będziemy prowadzili już bez key loggera, starając się korzystać jedynie z danych zapisanych w pliku. Oczywiście, jeśli zajdzie taka konieczność, można w każdej chwili wygenerować sobie nowy plik.

### Ćwiczenie 2: Napisz program, który wyliczy średnią prędkość pisania. Wykryj, kiedy użytkownik zaczął pisać. Nie bierz pod uwagę przerw dłuższych niż 5 sekund. Podaj prędkość pisania w znakach na minutę oraz słowach na minutę.

In [ ]:
from string import ascii_lowercase

def calculate_typing_speed():
    chars = list(ascii_lowercase)
    file = open('plik.tsv', 'r')
    times = []
    wordcount = 0
    for line in file.readlines():
        char, time = line.split('\t')
        time = float(time.strip())
        if char.lower() in chars or char == 'space':
            times.append(time)
        if char == 'space':
            wordcount += 1
    durations = []
    for index, time in enumerate(times):
        if index > 0:
            duration = time - prev_time
            if duration < 5.0:
                durations.append(duration)
        prev_time = time
    
    type_speed = len(durations) / sum(durations) 
    word_speed = wordcount / sum(durations)
    return type_speed * 60, word_speed * 60

calculate_typing_speed()

(173.1667608550016, 25.68502194500024)

Wróćmy teraz do procesu tłumaczenia. Analiza uderzeń klawiszy wykonanych podczas tłumaczenia pozwala wykryć dłuższe pauzy. Pauzy te najczęściej wskazują miejsca, w których tłumacz musi się głębiej zastanowić nad tłumaczeniem danego słowa lub frazy. Przerwę tę wykorzystuje na przykład na sprawdzenie tłumaczenia lub definicji w słowniku, przeglądanie wyników z pamięci tłumaczeń lub korzystanie z innych pomocy (eye-tracking mógłby w tym przypadku rozstrzygnąć, czym w istocie zajmuje się w tym momencie tłuamcz). Jest też możliwe, że tłumacz poświęca pauzę na tzw. cognitive pause-and-unload - rodzaj zamyślenia, pozwalający oczyścić myśli. Z punktu widzenia projektowania systemu wspomagającego tłumaczenie niezwykle istotna jest informacja, nad czym tłumacz musi się dłużej zastanowić. Minimalizacja liczby i czasu trwania takich przerw jest szansą na usprawnienie procesu tłumaczenia.

### Ćwiczenie 3: Napisz program do wykrywania przerw w pisaniu. Raportuj długość oraz miejsce wystąpienia przerwy podając 20-znakowy kontekst z każdej strony. Wykryj każdą przerwę dłuższą niż 3 sekundy, posortuj wyniki malejąco po długości przerwy.

In [ ]:
def find_pauses():
    char_list = list(ascii_lowercase)
    file = open('plik.tsv', 'r')
    times = []
    chars = []
    wordcount = 0
    for line in file.readlines():
        char, time = line.split('\t')
        time = float(time.strip())
        if char.lower() in char_list or char == 'space':
            times.append(time)
            chars.append(char)
    pauses = []
    for index, (time, char) in enumerate(zip(times, chars)):
        if index > 0:
            duration = time - prev_time
            if duration > 3.0:
                pauses.append((duration, '-'.join(chars[max(index - 20, 0) : index + 21])))
        prev_time = time
    return sorted(pauses, key=lambda x: x[0], reverse=True)
find_pauses()

[(8.74182677268982,
  'c-h-space-p-z-l-w-space-l-a-space-w-space-l-a-t-a-c-h-space-space-a-space-t-k-a-z-e-space-n-a-space-l-i-c-e-n-c-n-j-i'),
 (3.5999319553375244,
  'space-w-space-z-a-k-l-a-d-a-c-h-space-p-z-l-w-space-l-a-space-w-space-l-a-t-a-c-h-space-space-a-space-t-k-a-z-e-space-n-a'),
 (3.495737075805664,
  'k-i-space-s-a-m-o-l-o-t-space-m-y-l-w-i-s-k-i-space-k-o-n-s-t-r-u-c-k-j-i-space-i-n-z-y-n-i-e-r-a'),
 (3.032360553741455,
  'space-i-i-space-w-o-j-n-y-y-space-s-w-i-a-t-o-w-e-j-space-p-r-o-d-u-k-o-w-a-n-y-space-w-space-p-o-l-s-c-j')]